In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

We have get the results of sota solution DeOldify and need to see some metrics

In [3]:
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import lpips

def compute_metrics(label, prediction, data_range=255.0, lpips_f=lpips.LPIPS(net='vgg')):
    
    # Checking the shape
    if label.shape != prediction.shape:
        raise ValueError(f"Label shape({label.shape}) is incompatible with predicted shape({prediction.shape})")
    
    # Normalizing the images
    label_norm = label / data_range
    prediction_norm = prediction / data_range
    
    # PSNR metric
    psnr_val = psnr(label_norm, prediction_norm, data_range=data_range)
    
    #SSIM metric
    ssim_val = ssim(label_norm, prediction_norm, multichannel=True, channel_axis=-1, data_range=data_range)
    
    #LPIPS metric
    label_tensor = lpips.im2tensor(label_norm)
    prediction_tensor = lpips.im2tensor(prediction_norm)
    lpips_val = lpips_f(label_tensor, prediction_tensor).item()
    
    
    # np.array([psnr_val, ssim_val, lpips_val])
    return {
        'PSNR': psnr_val,
        'SSIM': ssim_val,
        'LPIPS': lpips_val
    }

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/vanishhhed/.pyenv/versions/ML_Ops/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vanishhhed/.pyenv/versions/ML_Ops/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/vanishhhed/.pyenv/versions/ML_Ops/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth


In [ ]:
# The results are in DeOldify_preds
if os.getcwd().split('/')[-1] != 'coco_subset':
    os.chdir('../dataset/coco_subset')

# print(os.getcwd())

file_names = [str(f).split('/')[1] for f in Path('val2017').glob('*.jpg')] #TODO после того как поменяется путь сохранения данных поменять на labels или inputs

results = {}

for file_name in tqdm(file_names, desc='Getting the result metrics of DeOldify'):
    # TODO change the name of the file after change in dataset_experiments
    try:
        label = cv2.imread('val2017/'+file_name) # TODO 'label/label_' + file_name
        colored = cv2.imread('DeOldify_preds/colored_input_' + file_name)
        
        # print(f"Label:\n\ttype: {type(label)}\t shape: {label.shape}\t values: {np.unique_values(label)[0]}->{np.unique_values(label)[-1]}")
        # print(f"Prediction:\n\ttype: {type(colored)}\t shape: {colored.shape}\t values: {np.unique_values(colored)[0]}->{np.unique_values(colored)[-1]}")
        
        results[file_name] = compute_metrics(label, colored)
        
    except Exception as e:
        print(f"ERROR: Exception: {e}\n in getting file:{file_name}")

results = pd.DataFrame(results)
print(results)

Getting the result metrics of DeOldify:   2%|▏         | 99/5000 [03:10<2:37:00,  1.92s/it]

       000000120584.jpg  000000501243.jpg  000000229601.jpg  000000448410.jpg  \
PSNR          68.674506         72.025455         64.432657         70.095148   
SSIM           0.998759          0.999539          0.996869          0.999270   
LPIPS          0.000109          0.000201          0.000356          0.000302   

       000000087875.jpg  000000044652.jpg  000000446005.jpg  000000521052.jpg  \
PSNR          66.372696         71.363640         73.565233         72.742883   
SSIM           0.997994          0.999325          0.999667          0.999664   
LPIPS          0.000251          0.000030          0.000092          0.000113   

       000000421757.jpg  000000013291.jpg  ...  000000015660.jpg  \
PSNR          76.898644         69.868542  ...         73.273052   
SSIM           0.999864          0.999265  ...          0.999766   
LPIPS          0.000045          0.000489  ...          0.000271   

       000000274219.jpg  000000574810.jpg  000000031817.jpg  000000482800.jpg

In [14]:
res = results.T
print(res.describe())

# results = pd.DataFrame(results.T)
# print(results[['PSNR', 'SSIM', 'LPIPS']].mean())

             PSNR        SSIM       LPIPS
count  100.000000  100.000000  100.000000
mean    71.873586    0.999308    0.000245
std      3.309040    0.000738    0.000261
min     63.395233    0.995967    0.000026
25%     69.832233    0.999158    0.000084
50%     72.080414    0.999558    0.000176
75%     73.665708    0.999743    0.000279
max     82.066901    0.999957    0.001637
